<a href="https://colab.research.google.com/github/paliwalpiyush151/Transfer-Learning-for-Catalysis/blob/main/Pre_training_general_domain_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Pre-training**
* This notebook contains the code for pre-training the general domain language model on 1 million molecules from the ChEMBL database using the SMILES representation
* The code is adapted from https://github.com/XinhaoLi74/MolPMoFiT/blob/master/notebooks/01_MSPM_Pretraining.ipynb

###*Install RdKit on Google Colaboratory*


In [ ]:
# Check which GPU is connected

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 25 08:36:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
 !git clone https://github.com/fastai/fastai1.git

Cloning into 'fastai1'...
remote: Enumerating objects: 32981, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 32981 (delta 96), reused 198 (delta 92), pack-reused 32769
Receiving objects: 100% (32981/32981), 471.68 MiB | 34.38 MiB/s, done.
Resolving deltas: 100% (23993/23993), done.
Checking out files: 100% (816/816), done.


In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.8 MB 1.5 MB/s 


In [ ]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO
'''
logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)

def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """

    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )

    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)

    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return

        logger.info("force re-install")

    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)

    logger.info("python version: {}".format(python_version))

    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)

    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')

    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')

    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python=={}".format(python_version),
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")

    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))


if __name__ == "__main__":
    install()

    '''

'\nlogger = getLogger(__name__)\nlogger.addHandler(StreamHandler())\nlogger.setLevel(INFO)\n\ndef install(\n        chunk_size=4096,\n        file_name="Miniconda3-latest-Linux-x86_64.sh",\n        url_base="https://repo.continuum.io/miniconda/",\n        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),\n        rdkit_version=None,\n        add_python_path=True,\n        force=False):\n    """install rdkit from miniconda\n    ```\n    import rdkit_installer\n    rdkit_installer.install()\n    ```\n    """\n\n    python_path = os.path.join(\n        conda_path,\n        "lib",\n        "python{0}.{1}".format(*sys.version_info),\n        "site-packages",\n    )\n\n    if add_python_path and python_path not in sys.path:\n        logger.info("add {} to PYTHONPATH".format(python_path))\n        sys.path.append(python_path)\n\n    if os.path.isdir(os.path.join(python_path, "rdkit")):\n        logger.info("rdkit is already installed")\n        if not force:\n            return\n

### Import Important Libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

from fastai1 import *
from fastai1.fastai.text import *
from fastai1.fastai.text.data import *

import numpy as np
import threading

##**Data**
Mount Google Colab to Google Drive to access the Data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load train and test data

In [ ]:
import pandas as pd
train = pd.read_csv('/content/gdrive/MyDrive/AI /Drug Discovery/TL_Catalysis_Code/Data/Pre-Training /ChemBL-LM_train.csv')
valid = pd.read_csv('/content/gdrive/MyDrive/AI /Drug Discovery/TL_Catalysis_Code/Data/Pre-Training /ChemBL-LM_val.csv')
train.shape, valid.shape

((980000, 2), (20000, 2))

In [ ]:
# Create a path to save the resluts
from pathlib import Path

result_path = Path('/content/gdrive/MyDrive/AI /Drug Discovery/TL_Catalysis_Code/results')
name = 'pre-trained_new'
path = result_path/name
path.mkdir(exist_ok=True, parents=True)

mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

# **SMILES augmentaion**

In [ ]:
def randomize_smiles(smiles):
    m = Chem.MolFromSmiles(smiles)
    ans = list(range(m.GetNumAtoms()))
    np.random.shuffle(ans)
    nm = Chem.RenumberAtoms(m,ans)
    return Chem.MolToSmiles(nm, canonical=False, isomericSmiles=True, kekuleSmiles=False)

def smiles_augmentation(df, N_rounds):
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['SMILES'].append(randomize_smiles(df.iloc[i].SMILES))
            dist_aug['canonical'].append('no')

    df_aug = pd.DataFrame.from_dict(dist_aug)

    #merge with original df
    df = pd.concat([df, df_aug], sort=False).reset_index(drop=True)
    #shuffle the data
    df = df.reindex(np.random.permutation(df.index))
    return pd.DataFrame.from_dict(df).drop_duplicates('SMILES')

The randomized SMILES are used for data augmentation. Each SMILES is augmented by four other SMILES

In [ ]:
%%time
train_aug = smiles_augmentation(train, 4)
valid_aug = smiles_augmentation(valid, 4)

# I have changed it to 1 from 4
# It took 33 mins to finish this task.

CPU times: user 32min 54s, sys: 11.1 s, total: 33min 5s
Wall time: 32min 57s


##**Data pre-processing**
Define a custom tokenizer

In [ ]:
# Don't include the defalut specific token of fastai, only keep the padding token

BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'
defaults.text_spec_tok = [PAD]

special_tokens = ['[BOS]', '[C@H]', '[C@@H]','[C@]', '[C@@]','[C-]','[C+]', '[c-]', '[c+]','[cH-]',
                   '[nH]', '[N+]', '[N-]', '[n+]', '[n-]' '[NH+]', '[NH2+]',
                   '[O-]', '[S+]', '[s+]', '[S-]', '[O+]', '[SH]', '[B-]','[BH2-]', '[BH3-]','[b-]',
                   '[PH]','[P+]', '[I+]',
                  '[Si]','[SiH2]', '[Se]','[SeH]', '[se]', '[Se+]', '[se+]','[te]','[te+]', '[Te]']

class MolTokenizer(BaseTokenizer):
    def __init__(self, lang = 'en', special_tokens = special_tokens):
        self.lang = lang
        self.special_tokens = special_tokens

    def tokenizer(self, smiles):
        # add specific token '[BOS]' to represetences the start of SMILES
        smiles = '[BOS]' + smiles
        regex = '(\[[^\[\]]{1,10}\])'
        char_list = re.split(regex, smiles)
        tokens = []

        if self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    if char in special_tokens:
                        tokens.append(str(char))
                    else:
                        tokens.append('[UNK]')
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]

        if not self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    tokens.append(str(char))
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]

        #fix the 'Br' be splited into 'B' and 'r'
        if 'B' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'B':
                    if index < len(tokens)-1: # make sure 'B' is not the last character
                        if tokens[index+1] == 'r':
                            tokens[index: index+2] = [reduce(lambda i, j: i + j, tokens[index : index+2])]

        #fix the 'Cl' be splited into 'C' and 'l'
        if 'l' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'l':
                    if tokens[index-1] == 'C':
                            tokens[index-1: index+1] = [reduce(lambda i, j: i + j, tokens[index-1 : index+1])]
        return tokens

    def add_special_cases(self, toks):
        pass

In [ ]:
# Tokenizer
tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])

Create a text databunch for language modeling:

* It takes as input the train and validation data
* Pass the custom tokenizer defined in the previous step
* Specify the column containing text data
* Define the batch size according to the GPU memory available

In [ ]:
%%time
bs = 128  #batch size

data = TextLMDataBunch.from_df(path, train_aug, valid_aug, bs=bs, tokenizer=tok,
                               chunksize=50000, text_cols=0, max_vocab=60000, include_bos=False)

/content/fastai1/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


CPU times: user 1min 39s, sys: 1min 24s, total: 3min 4s
Wall time: 3min 18s


In [ ]:
%%time
# Save the databunch
data.save(f'{name}_databunch')

# databunch : input data for neural network feeding

CPU times: user 1min 58s, sys: 9.53 s, total: 2min 7s
Wall time: 2min 15s


##**Training the model**

Load the data bunch

In [ ]:
bs = 128 # batch size
data_lm = load_data(path, f'{name}_databunch', bs=bs)

Create a learner for language modeling:

* As the model is trained from scratch, use pretrained=False
* Pass the text databunch loaded in the previous step
* Drop_mult is a hyperparameter that can be tuned

In [ ]:
learner = language_model_learner(data_lm, AWD_LSTM, drop_mult=1, pretrained=False)

In [ ]:
# Model architecture
learner.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(80, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(80, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=80, bias=True)
    (output_dp): RNNDropout()
  )
)

Train the model using fit_one_cycle

* the first hyperparameter is number of epochs
* the second hyperparameter is learning rate

In [ ]:
#lr = 3e-3
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

learner.unfreeze()
learner.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.828258,0.772966,0.727074,1:25:42
1,0.889364,0.829432,0.710240,1:25:42
2,0.904451,0.832295,0.708657,1:25:38


KeyboardInterrupt: ignored

Save the weights and vocabulary of the trained model

In [ ]:
lm_fns = [f'{name}_wt', f'{name}_vocab']

learner.save(lm_fns[0], with_opt=False)
learner.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

## How to use colab pro?
### https://colab.research.google.com/notebooks/pro.ipynb#scrollTo=QMMqmdiYMkvi